In [5]:
import akshare as ak
import tqdm
import pandas as pd
import datetime
cdf = pd.read_csv("./daily/stock_hist.csv")
lst = []
hdf = pd.read_csv("./daily/stock_fund_hist.csv")
hs = set(hdf["code"].drop_duplicates())
cdf = cdf[~cdf.code.isin(hs)]
lst.append(hdf)

def fetch_stock_fund():
    bar = tqdm.tqdm(total=cdf.shape[0],desc="Fetch stock fund data:")
    for i,v in cdf.iterrows():
        if i>5000:
            break
        code = v["code"]
        sc,sm = code[2:],code[:2]
        bar.update()
        try:
            tdf = ak.stock_individual_fund_flow(stock=sc, market=sm)
            tdf[["code","name"]]=(v["code"],v["name"])
            tdf = tdf.rename(columns={"日期":"date","主力净流入-净额":"main_amount","主力净流入-净占比":"main_percent"})
            tdf = tdf[["date","code","main_amount","main_percent"]]
            tdf = tdf[tdf["date"]>=datetime.datetime.strptime("2024-11-15",'%Y-%m-%d').date()]
            lst.append(tdf)
        except:
            import traceback
            traceback.print_exc()
            print(v)
    if len(lst)>1:
        df = pd.concat(lst)
        df.to_csv("stock_fund.csv",index=False)
# fetch_stock_fund()

In [ ]:
import akshare as ak
import datetime
import pandas as pd

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True


def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code

def reform_main(v):
    if v=="-":
        return 0.0
    else:
        return float(v)
    
    
today,bias = datetime.datetime.now(),1
cdate, pdate = today+datetime.timedelta(days=-bias),today+datetime.timedelta(days=-bias-1)
cdate= cdate.strftime('%Y-%m-%d')
cfund_df = ak.stock_individual_fund_flow_rank(indicator="今日")
cdf = cfund_df.rename(columns={"代码":"code","名称":"name","今日主力净流入-净额":"main_amount","今日主力净流入-净占比":"main_percent"})
columns = ["date","code","main_amount","main_percent"]
cdf = cdf[cdf.apply(filter_shsz,axis=1)]
cdf["code"] = cdf["code"].apply(reform_shsz)
cdf["date"]=cdate
cdf = cdf[columns+["name"]]
cdf["main_amount"] = cdf["main_amount"].apply(reform_main)
cdf["main_percent"] = cdf["main_percent"].apply(reform_main)
cdf[columns].to_csv("stock_fund_today.csv",index=False)
cdf.sort_values("main_amount",ascending=False)

/Users/john/anaconda3/lib/python3.11/site-packages/akshare/__init__.py:2977: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.2.0 及以上版本！
  warnings.warn(


In [3]:
import pandas as pd

def save_dt():
    def get_weekday(cdate):
        holidays = set(["2025-01-01"])
        for i in range(1,5):
            ttm = today+datetime.timedelta(days=-bias-i)
            if ttm.weekday() <5 and ttm.strftime("%Y-%m-%d") not in holidays:
                return ttm

    today = datetime.datetime.now()
    cdate = today+datetime.timedelta(days=-bias)
    pdate = get_weekday(cdate)
    cdate,pdate = cdate.strftime('%Y-%m-%d'),pdate.strftime('%Y-%m-%d')
    df = pd.read_csv(f"daily/stock_fund_{pdate}.csv")
    df=pd.concat([df,cdf])
    df["main_amount"] = df["main_amount"].apply(reform_main)
    df["main_percent"] = df["main_percent"].apply(reform_main)
    df[columns].to_csv(f"daily/stock_fund_{cdate}.csv",index=False)
    cdf[columns].to_csv(f"daily/stock_fund_daily_{cdate}.csv",index=False)

def clean_data(pattern="stock_fund_\d{4}-\d{2}-\d{2}.csv"):
    import os, re
    lst = []
    for r,d, files in os.walk("./daily"):
        for f in files:
            if re.match(pattern,f):
                lst.append(os.path.join(r,f))
    for f in sorted(lst)[:-2]:
        os.remove(f)

save_dt()
clean_data()